In [ ]:
import pandas as pd
import random

In [ ]:
from datasets import load_dataset

ds = load_dataset("rasdani/cohere-wikipedia-2023-11-da")

In [ ]:
df = ds["train"].to_pandas()

In [ ]:
df.shape

(1224982, 4)

In [ ]:
df.head()

,_id,url,title,text
0,20231101.da_508659_0,https://da.wikipedia.org/wiki/Piccadilly%20Cir...,Piccadilly Circus Station,Piccadilly Circus Station er en London Undergr...
1,20231101.da_508659_1,https://da.wikipedia.org/wiki/Piccadilly%20Cir...,Piccadilly Circus Station,Station blev åbnet den 10. marts 1906 af Baker...
2,20231101.da_508659_2,https://da.wikipedia.org/wiki/Piccadilly%20Cir...,Piccadilly Circus Station,"Den gamle stationsbygningen, designet af Lesli..."
3,20231101.da_508659_3,https://da.wikipedia.org/wiki/Piccadilly%20Cir...,Piccadilly Circus Station,Bakerloo line-perronerne på Piccadilly Circus ...
4,20231101.da_508659_4,https://da.wikipedia.org/wiki/Piccadilly%20Cir...,Piccadilly Circus Station,Piccadilly Circus er foreslået som stop på Che...


In [ ]:
# For memory management, I will select 100 of those areticles with >9 paragraphs

filtered_articles = df.groupby('title').filter(lambda x: len(x) >= 9)
selected_titles = filtered_articles['title'].drop_duplicates().sample(n=100, random_state=42)
selected_articles = filtered_articles[filtered_articles['title'].isin(selected_titles)]

In [ ]:
import random

processed_samples = []

"""The function below will do the following:
Group paragraphs by article title and sort if needed.
Check that each article has at least 9 paragraphs; skip otherwise.
Randomly select a starting index for a 9-paragraph window.
Use the middle (5th) paragraph as the positive sample; surrounding paragraphs are negatives.
Create a sample dictionary, using the article ID and URL from the first row.
Append the processed sample to a list.
"""

for title, group in df.groupby('title'):
    paragraphs = group['text'].tolist()
    if len(paragraphs) < 9:
        continue
    start_idx = random.randint(0, len(paragraphs) - 9)
    selected_window = paragraphs[start_idx:start_idx + 9]
    middle_index = 4
    positive_sample = selected_window[middle_index]
    negative_samples = selected_window[:middle_index] + selected_window[middle_index + 1:]

    sample = {
        'article_id': group['_id'].iloc[0],
        'title': title,
        'url': group['url'].iloc[0],
        'positive': positive_sample,
        'negatives': negative_samples
    }

    processed_samples.append(sample)

print("Processing complete. Number of samples:", len(processed_samples))


Processing complete. Number of samples: 30280


In [ ]:
processed_df = pd.DataFrame(processed_samples)

In [ ]:
processed_df.head()

,article_id,title,url,positive,negatives
0,20231101.da_470802_0,'Ndrangheta,https://da.wikipedia.org/wiki/%27Ndrangheta,I de senere år har organisationen søgt mod Nor...,[Indtil midten af 1970’erne var ‘Ndrangheta en...
1,20231101.da_282955_0,(2) Pallas,https://da.wikipedia.org/wiki/%282%29%20Pallas,Der har endnu ikke været nogle teleskopobserva...,[Under sin søgen efter Ceres bemærkede Olbers ...
2,20231101.da_282878_0,(3) Juno,https://da.wikipedia.org/wiki/%283%29%20Juno,S-type asteroiderne er kendt for at have et la...,[Radiosignaler fra rumsonder i kredsløb om Mar...
3,20231101.da_235772_0,(4) Vesta,https://da.wikipedia.org/wiki/%284%29%20Vesta,Vestas kugleform brydes markant af et 460 kilo...,[Vestas facon afviger lidt fra den perfekte ku...
4,20231101.da_27766_0,(90377) Sedna,https://da.wikipedia.org/wiki/%2890377%29%20Sedna,"De fremstillede også en anden, mindre sandsynl...","[Man kender ikke Sednas præcise omløbstid, men..."


In [ ]:
#save
processed_df.to_parquet("processed_danish_wikipedia.parquet", index=False)

#Push To Hugging Face Hub

In [ ]:
from datasets import Dataset
from huggingface_hub import login


login()


In [ ]:
processed_dataset = Dataset.from_pandas(processed_df)

In [ ]:
processed_dataset.push_to_hub("meshachaderele/negative-positive-wikipedia-2023-11-da")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/31 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/meshachaderele/negative-positive-wikipedia-2023-11-da/commit/be1ea6bcabba9bc704a54d000a7a3c1a59c0a79a', commit_message='Upload dataset', commit_description='', oid='be1ea6bcabba9bc704a54d000a7a3c1a59c0a79a', pr_url=None, pr_revision=None, pr_num=None)